In [2]:
import numpy as np
import pandas as pd

In [3]:
in_path = "../data/processed/user_logs_aggregated.parquet"
out_path = "../data/processed/user_logs_aggregated_optimized.parquet"

df = pd.read_parquet(in_path)

print("Before")
print(df.info(memory_usage="deep"))

# -----------------------
# 1) msno: string으로
# -----------------------
if "msno" in df.columns:
    try:
        df["msno"] = df["msno"].astype("string[pyarrow]")
    except Exception:
        df["msno"] = df["msno"].astype("string")

# -----------------------
# 2) 컬럼 패턴 기반 타입 지정
# -----------------------
# float32로 둘 컬럼 (연속값)
float32_patterns = (
    "total_secs_",
    "avg_secs_",
    "std_secs_",
    "skip_ratio_",
    "completion_ratio_",
    "short_play_ratio_",
    "variety_ratio_",
    "_trend_",
    "recency_",
)

# 정수로 둘 컬럼 (count/일수)
# - num_days_active_* 는 0~31 수준
# - num_* / short_play_* / num_songs_* / num_unq_* 등 count성
int_like_prefixes = (
    "num_days_active_",
    "num_songs_",
    "num_unq_",
    "num_25_",
    "num_100_",
    "short_play_",
)

def is_float32_target(col: str) -> bool:
    return any(p in col for p in float32_patterns)

def is_int_like_target(col: str) -> bool:
    return col.startswith(int_like_prefixes)

# -----------------------
# 3) int-like 컬럼: "정수로 깔끔히 변환 가능하면" UInt로 내리기
#    (소수/비정상/결측 있으면 float32 유지)
# -----------------------
for c in df.columns:
    if c == "msno":
        continue

    s = df[c]

    # float32 강제 대상
    if is_float32_target(c):
        df[c] = pd.to_numeric(s, errors="coerce").astype(np.float32)
        continue

    # int-like 후보
    if is_int_like_target(c):
        # float에서 온 경우가 많아서: 정수인지 체크 (결측 제외)
        s_num = pd.to_numeric(s, errors="coerce")
        non_na = s_num.dropna()

        # 전부 정수 형태면 (예: 6.0, 31.0) -> int로
        if len(non_na) == 0:
            # 전부 NA면 nullable UInt16로 (아무거나 하나)
            df[c] = s_num.astype("UInt16")
            continue

        is_integerish = np.isclose(non_na.values, np.round(non_na.values)).all()

        if is_integerish:
            # 값 범위 보고 최적 UInt 선택
            mn = int(non_na.min())
            mx = int(non_na.max())

            # 음수면 UInt 못 씀 -> Int로
            if mn < 0:
                # 크기별 downcast
                df[c] = pd.to_numeric(np.round(s_num), errors="coerce", downcast="integer")
            else:
                # nullable UInt로 캐스팅 (결측 지원)
                if mx <= np.iinfo(np.uint8).max:
                    df[c] = pd.array(np.round(s_num), dtype="UInt8")
                elif mx <= np.iinfo(np.uint16).max:
                    df[c] = pd.array(np.round(s_num), dtype="UInt16")
                elif mx <= np.iinfo(np.uint32).max:
                    df[c] = pd.array(np.round(s_num), dtype="UInt32")
                else:
                    df[c] = pd.array(np.round(s_num), dtype="UInt64")
        else:
            # 정수형으로 못 내리면 float32로 최소화
            df[c] = s_num.astype(np.float32)

# -----------------------
# 4) 나머지 숫자 컬럼: downcast
#    (여기서 남는 건 대부분 float64일 가능성)
# -----------------------
for c in df.columns:
    if c == "msno":
        continue
    if pd.api.types.is_float_dtype(df[c]):
        df[c] = pd.to_numeric(df[c], errors="coerce", downcast="float")
    elif pd.api.types.is_integer_dtype(df[c]):
        df[c] = pd.to_numeric(df[c], errors="coerce", downcast="integer")

print("\nAfter")
print(df.info(memory_usage="deep"))

# -----------------------
# 5) 저장 (zstd 추천: 용량 잘 줄어듦)
# -----------------------
df.to_parquet(
    out_path,
    index=False,
    engine="pyarrow",
    compression="zstd",
)

print(f"\nSaved: {out_path}")

Before
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103894 entries, 0 to 1103893
Data columns (total 67 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   msno                     1103894 non-null  object 
 1   num_days_active_w7       1103894 non-null  float64
 2   total_secs_w7            1103894 non-null  float64
 3   avg_secs_per_day_w7      1103894 non-null  float64
 4   std_secs_w7              1103894 non-null  float64
 5   num_songs_w7             1103894 non-null  float64
 6   avg_songs_per_day_w7     1103894 non-null  float64
 7   num_unq_w7               1103894 non-null  float64
 8   num_25_w7                1103894 non-null  float64
 9   num_100_w7               1103894 non-null  float64
 10  short_play_w7            1103894 non-null  float64
 11  skip_ratio_w7            1103894 non-null  float64
 12  completion_ratio_w7      1103894 non-null  float64
 13  short_play_ratio_w7      1103894 no